In [2]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
%matplotlib inline
import _pickle
import time

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# from alexnet_backprop import *
from alexnet_guided_bp_vanilla import *
from utils import *

In [5]:
# load test set images
test_images = open('test_set_17_227.pkl', "rb")
data_set = _pickle.load(test_images) 
actor_code = get_actor_code(data_set)[0]
reversed_actor_code = get_actor_code(data_set)[1]

# load weights
alexnet_weight_file = 'alexnet_weights.pkl'

weight_fpath = 'tl17_result_trial2/'
weight_fname = 'transfer_learning_fc_weights_17_trial2.pkl' 
tl_weight_file = weight_fpath + weight_fname

weight_fpath = 'e2e17_result_trial2/'
weight_fname = 'end_to_end_learning_weights_17_trial2.pkl' 
e2e_weight_file = weight_fpath + weight_fname

weight_fpath = 'e2e17_result_trial2_ld/'
weight_fname = 'end_to_end_learning_weights_17_trial2_ld.pkl' 
lessdecay_e2e_weight_file = weight_fpath + weight_fname

In [5]:
def generate_true_class_saliency(data_set, graph, sess, fname):
    
    # save saliencies in an external array
    saliency_masks = {}
    for actor in data_set:
        saliency_masks[actor] = np.zeros((0,227,227))
    saliency_masked = {}
    for actor in data_set:
        saliency_masked[actor] = np.zeros((0,227,227,3))
    
    
    plt.figure(dpi=70, figsize=(140, 100))
    col=0
    for actor in data_set:
        row = 0 
        for _ in range(len(data_set[actor])):
            
            # forward pass
            image = data_set[actor][_].astype(np.float64)
            image_feed = np.expand_dims(image,0)
            probabilities = sess.run(graph.probabilities, feed_dict={graph.inputs:image_feed})[0]
    
            # plot original image
            index = row*len(data_set)*2 + col +1
            plt.subplots_adjust(bottom = 0., wspace=0.15, hspace = 0.1, top=0.5)
            plt.subplot(len(data_set[actor]), len(data_set)*2, index)
            plt.axis('off')
            plt.title("True: " + actor + "\n Pred: " + actor_code[np.argmax(probabilities)], fontsize=10)
            plt.imshow(image.astype('uint8'))
    
            # plot true class_saliency
            i = reversed_actor_code[actor]
            one_hot = np.zeros(len(actor_code))
            one_hot[i] = 1
            plt.subplot(len(data_set[actor]), len(data_set)*2, index+1)
            plt.axis('off')
            plt.title(actor + '\n' + str(probabilities[i]), fontsize=14)
            detailed_saliency = guided_backprop(graph, image, one_hot, sess)
            saliency = np.sum(detailed_saliency, axis=-1)
            saliency_rank = saliency.ravel().argsort().argsort().reshape(saliency.shape)
            saliency_mask = (saliency_rank > 227*227*0.95).astype(float)
            masked_saliency = np.expand_dims(saliency_mask, axis=-1)*detailed_saliency
            
            plt.imshow(masked_saliency)

            saliency_masked[actor] = np.vstack((saliency_masked[actor], np.expand_dims(masked_saliency, axis=0)))
            saliency_masks[actor] = np.vstack((saliency_masks[actor], np.expand_dims(saliency_mask, axis=0)))
            row += 1
        col += 2
    plt.savefig(fname +'.pdf', bbox_inches='tight')
    plt.close()
    
    pickle_out = open(fname +".pkl","wb")
    _pickle.dump(saliency_masked, pickle_out)

In [6]:
# hyperparameters and backprop variants
tau = 1.0

In [76]:
tf.reset_default_graph()
b_graph = backprop_graph(17, 100, alexnet_face_classifier)
b_graph.classifier_graph(temp=tau)
b_graph.guided_backprop_graph()

In [77]:
try:
    os.mkdir('weight comparison')
except:
    pass

In [78]:
# first generate the saliency using original alexnet + tl weights
with tf.Session() as sess:
    b_graph.cnn.load_weights(alexnet_weight_file, sess, conv_only = True)
    b_graph.cnn.load_weights(tl_weight_file, sess, fc_only = True)
    generate_true_class_saliency(data_set, b_graph, sess, 'weight comparison/tl_saliency_robust_detail')

0 conv1W (11, 11, 3, 96)
1 conv1b (96,)
2 conv2W (5, 5, 48, 256)
3 conv2b (256,)
4 conv3W (3, 3, 256, 384)
5 conv3b (384,)
6 conv4W (3, 3, 192, 384)
7 conv4b (384,)
8 conv5W (3, 3, 192, 256)
9 conv5b (256,)
0 fc1W (43264, 100)
1 fc1b (100,)
2 fc2W (100, 17)
3 fc2b (17,)


In [79]:
with tf.Session() as sess:
    b_graph.cnn.load_weights(e2e_weight_file, sess)
    generate_true_class_saliency(data_set, b_graph, sess, 'weight comparison/e2e_saliency_robust_detail')

0 conv1W (11, 11, 3, 96)
1 conv1b (96,)
2 conv2W (5, 5, 48, 256)
3 conv2b (256,)
4 conv3W (3, 3, 256, 384)
5 conv3b (384,)
6 conv4W (3, 3, 192, 384)
7 conv4b (384,)
8 conv5W (3, 3, 192, 256)
9 conv5b (256,)
10 fc1W (43264, 100)
11 fc1b (100,)
12 fc2W (100, 17)
13 fc2b (17,)


In [5]:
with open(alexnet_weight_file, 'rb') as handle:
    alexnet_weights = _pickle.load(handle)

with open(e2e_weight_file, 'rb') as handle:
    e2e_weights = _pickle.load(handle)

# with open(lessdecay_e2e_weight_file, 'rb') as handle:
#     lessdecay_e2e_weights = _pickle.load(handle)

In [6]:
alexnet_weights.keys()

dict_keys(['conv1W', 'conv1b', 'conv2W', 'conv2b', 'conv3W', 'conv3b', 'conv4W', 'conv4b', 'conv5W', 'conv5b', 'fc1W', 'fc1b', 'fc2W', 'fc2b', 'fc3W', 'fc3b'])

In [7]:
threshold = 0.005
for layer in ['conv1W', 'conv2W', 'conv3W', 'conv4W', 'conv5W']:
    print(layer)
    print(np.mean(np.absolute(alexnet_weights[layer])<threshold))
    
for layer in ['conv1W', 'conv2W', 'conv3W', 'conv4W', 'conv5W']:
    print(layer)
    print(np.mean(np.absolute(e2e_weights[layer])<threshold))

conv1W
0.132202708907
conv2W
0.252965494792
conv3W
0.287393075448
conv4W
0.240549346547
conv5W
0.224932635272
conv1W
0.159750918274
conv2W
0.503684895833
conv3W
0.664001464844
conv4W
0.6518720462
conv5W
0.734985351562


In [8]:
# n, bins, patches = plt.hist(np.absolute(e2e_weights['conv3W']).flatten(),\
#                             40, facecolor='g', alpha=0.75)
# plt.xlabel('Absolute Weight')
# plt.ylabel('Proportion')
# plt.title('Conv1 Weights')
# # plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
# # plt.axis([40, 160, 0, 0.03])
# plt.grid(True)
# plt.show()

In [15]:
def generate_true_class_saliency_comparison_samples(data_set, sample_list, tl_saliencies, e2e_saliencies, 
                                        fname, num_col_splits = 2):
        
    
    f, axes = plt.subplots(len(sample_list)//num_col_splits, 3*num_col_splits, figsize=(80,26))
    plt.subplots_adjust(wspace=0.02, hspace = 0.02)
    col_splits = [0+i*3 for i in range(num_col_splits)]
    
#     from matplotlib import rcParams
#     rcParams['axes.titlepad'] = 20 
    
    for col in col_splits:
        axes[0][col].set_title("Original Image", fontsize=45)
        axes[0][col].title.set_position([.5, 1.02])
        axes[0][col+1].set_title("Transfer Learning", fontsize=45)
        axes[0][col+1].title.set_position([.5, 1.02])
        axes[0][col+2].set_title("End-to-end Learning", fontsize=45)
        axes[0][col+2].title.set_position([.5, 1.02])



    
    actor_count = 0
    for col in col_splits:
        for row in range(len(sample_list)//len(col_splits)):
            actor, index = sample_list[actor_count]
            axes[row][col].axis('off')
            axes[row][col].imshow(data_set[actor][index].astype('uint8'))
            
            axes[row][col+1].axis('off')
            axes[row][col+1].imshow(tl_saliencies[actor][index])
            
            axes[row][col+2].axis('off')
            axes[row][col+2].imshow(e2e_saliencies[actor][index])
            
            actor_count += 1
        
    plt.savefig('weight comparison/' + fname +'.pdf', bbox_inches='tight')
    plt.close()
    

In [16]:
# samples
# samples = {'Anne Hathaway': [2,5,7], 'Jennifer Aniston':[1,2,14], 'America Ferrera':[0,4,14],
#           'Nicolas Cage':[1,3,12], 'Gerard Butler':[3,7,14], 'Daniel Radcliffe':[2,13,14]}
# samples = {'America Ferrera':[0,4,14], 'Gerard Butler':[3,7,14]}
samples = {'America Ferrera':[0,14], 'Gerard Butler':[3,14]}
sample_list = []
for actor in samples:
    for index in samples[actor]:
        sample_list.append((actor, index))

In [6]:
with open('weight comparison/tl_saliency_robust_detail.pkl', 'rb') as handle:
    tl_saliencies = _pickle.load(handle)
with open('weight comparison/e2e_saliency_robust_detail.pkl', 'rb') as handle:
    e2e_saliencies = _pickle.load(handle)

In [18]:
generate_true_class_saliency_comparison_samples(data_set, sample_list, tl_saliencies, e2e_saliencies, 'tl_vs_e2e')

In [34]:
# good_exps = [('Daniel Radcliffe', 0), ('Daniel Radcliffe', 3), ('Anne Hathaway', 5), ('Anne Hathaway', 7),
#              ('Angie Harmon', 1), ('Angie Harmon', 14), ('Fran Drescher', 4), ('Fran Drescher', 2)]
# bad_exps = [('Jennifer Aniston', 1), ('Jennifer Aniston', 2), ('Bill Hader', 2), ('Bill Hader', 3)]

# num_col_splits = 2


# fname = 'good_exps'


# f, axes = plt.subplots(len(good_exps)//2, 2*num_col_splits, figsize=(65,65))

# plt.subplots_adjust(wspace=0.02, hspace = 0.02)
# col_splits = [0+i*2 for i in range(num_col_splits)]
# sample_list = good_exps
# #     from matplotlib import rcParams
# #     rcParams['axes.titlepad'] = 20 

# for col in col_splits:
#     axes[0][col].set_title("Original Image", fontsize=45)
#     axes[0][col+1].set_title("Saliency Map", fontsize=45)

# actor_count = 0
# for col in col_splits:
#     for row in range(len(sample_list)//len(col_splits)):
#         actor, index = sample_list[actor_count]
#         axes[row][col].axis('off')
#         axes[row][col].imshow(data_set[actor][index].astype('uint8'))

#         axes[row][col+1].axis('off')
#         axes[row][col+1].imshow(e2e_saliencies[actor][index])

#         actor_count += 1

# plt.savefig('weight comparison/' + fname +'.png', bbox_inches='tight')
# plt.close()

    
# fname = 'bad_exps'
# f, axes = plt.subplots(len(bad_exps)//2, 2*num_col_splits, figsize=(65,32.5))
# plt.subplots_adjust(wspace=0.02, hspace = 0.02)
# col_splits = [0+i*2 for i in range(num_col_splits)]
# sample_list = bad_exps
# #     from matplotlib import rcParams
# #     rcParams['axes.titlepad'] = 20 

# for col in col_splits:
#     axes[0][col].set_title("Original Image", fontsize=45)
#     axes[0][col+1].set_title("Saliency Map", fontsize=45)

# actor_count = 0
# for col in col_splits:
#     for row in range(len(sample_list)//len(col_splits)):
#         actor, index = sample_list[actor_count]
#         axes[row][col].axis('off')
#         axes[row][col].imshow(data_set[actor][index].astype('uint8'))

#         axes[row][col+1].axis('off')
#         axes[row][col+1].imshow(e2e_saliencies[actor][index])

#         actor_count += 1

# plt.savefig('weight comparison/' + fname +'.png', bbox_inches='tight')
# plt.close()


In [14]:
exps = [('Daniel Radcliffe', 0), ('Angie Harmon', 1), ('Anne Hathaway', 5), ('Fran Drescher', 2), 
        ('Jennifer Aniston', 1), ('Bill Hader', 3)]

num_col_splits = 3


fname = 'exps'


f, axes = plt.subplots(len(exps)//num_col_splits, 2*num_col_splits, figsize=(80,26))

plt.subplots_adjust(wspace=0.02, hspace = 0.02)
col_splits = [0+i*2 for i in range(num_col_splits)]
sample_list = exps
#     from matplotlib import rcParams
#     rcParams['axes.titlepad'] = 20 

for col in col_splits:
    axes[0][col].set_title("Original Image", fontsize=45)
    axes[0][col].title.set_position([.5, 1.02])
    axes[0][col+1].set_title("Saliency Map", fontsize=45)
    axes[0][col+1].title.set_position([.5, 1.02])

actor_count = 0
for col in col_splits:
    for row in range(len(sample_list)//len(col_splits)):
        actor, index = sample_list[actor_count]
        axes[row][col].axis('off')
        axes[row][col].imshow(data_set[actor][index].astype('uint8'))

        axes[row][col+1].axis('off')
        axes[row][col+1].imshow(e2e_saliencies[actor][index])

        actor_count += 1

plt.savefig('weight comparison/' + fname +'.pdf', bbox_inches='tight')
plt.close()

In [ ]:
def generate_all_saliency_samples(e2e_saliencies):
    
    # save saliencies in an external array
    saliency_masked = {}
    for actor in data_set:
        saliency_masked[actor] = np.zeros((0,227,227,3))
    
    
    plt.figure(dpi=70, figsize=(140, 100))
    col=0
    for actor in data_set:
        row = 0 
        for _ in range(len(data_set[actor])):
            
            # forward pass
            image = data_set[actor][_].astype(np.float64)
            image_feed = np.expand_dims(image,0)
            probabilities = sess.run(graph.probabilities, feed_dict={graph.inputs:image_feed})[0]
    
            # plot original image
            index = row*len(data_set)*2 + col +1
            plt.subplots_adjust(bottom = 0., wspace=0.15, hspace = 0.1, top=0.5)
            plt.subplot(len(data_set[actor]), len(data_set)*2, index)
            plt.axis('off')
            plt.title("True: " + actor + "\n Pred: " + actor_code[np.argmax(probabilities)], fontsize=10)
            plt.imshow(image.astype('uint8'))
    
            # plot true class_saliency
            i = reversed_actor_code[actor]
            one_hot = np.zeros(len(actor_code))
            one_hot[i] = 1
            plt.subplot(len(data_set[actor]), len(data_set)*2, index+1)
            plt.axis('off')
            plt.title(actor + '\n' + str(probabilities[i]), fontsize=14)
            detailed_saliency = guided_backprop(graph, image, one_hot, sess)
            saliency = np.sum(detailed_saliency, axis=-1)
            saliency_rank = saliency.ravel().argsort().argsort().reshape(saliency.shape)
            saliency_mask = (saliency_rank > 227*227*0.95).astype(float)
            masked_saliency = np.expand_dims(saliency_mask, axis=-1)*detailed_saliency
            
            plt.imshow(masked_saliency)

            saliency_masked[actor] = np.vstack((saliency_masked[actor], np.expand_dims(masked_saliency, axis=0)))
            saliency_masks[actor] = np.vstack((saliency_masks[actor], np.expand_dims(saliency_mask, axis=0)))
            row += 1
        col += 2
    plt.savefig(fname +'.pdf', bbox_inches='tight')
    plt.close()
    
    pickle_out = open(fname +".pkl","wb")
    _pickle.dump(saliency_masked, pickle_out)

In [17]:
def generate_all_saliency_samples(actor, data_set, e2e_saliencies, num_col_splits = 3):
        
    
    f, axes = plt.subplots(5, 2*num_col_splits, figsize=(80,70))
    plt.subplots_adjust(wspace=0.02, hspace = 0.02)
    col_splits = [0+i*2 for i in range(num_col_splits)]
    
#     from matplotlib import rcParams
#     rcParams['axes.titlepad'] = 20 
    
#     for col in col_splits:
#         axes[0][col].set_title("Original Image", fontsize=45)
#         axes[0][col].title.set_position([.5, 1.02])
#         axes[0][col+1].set_title("Transfer Learning", fontsize=45)
#         axes[0][col+1].title.set_position([.5, 1.02])
#         axes[0][col+2].set_title("End-to-end Learning", fontsize=45)
#         axes[0][col+2].title.set_position([.5, 1.02])

    
    index = 0
    for col in col_splits:
        for row in range(5):
#             actor, index = sample_list[actor_count]
            axes[row][col].axis('off')
            axes[row][col].imshow(data_set[actor][index].astype('uint8'))
            
            axes[row][col+1].axis('off')
            axes[row][col+1].imshow(e2e_saliencies[actor][index])
            
            index += 1
        
    plt.savefig('weight comparison/' + actor +'.pdf', bbox_inches='tight')
    plt.close()

In [19]:
for actor in data_set:
    generate_all_saliency_samples(actor, data_set, e2e_saliencies, num_col_splits = 3)